# Instalação

In [ ]:
!pip install xcompact3d-toolbox

# Lendo e salvando arquivos

Esse tutorial apresenta diferentes formas com que se pode carregar arquivos binários do disco, após rodar uma simulação com [XCompact3d](https://github.com/xcompact3d/Incompact3d). Além disso, são apresentados algumas opções para salvar resultados de pós-processamento.

## Preparação

Aqui se prepara o banco de dados para esse notebook, para que possa ser reproduzido na máquina local ou na nuvem.

O primeiro passo é para importar as bibliotecas para esse notebook.

In [1]:
import warnings

import numpy as np
import xarray as xr

import xcompact3d_toolbox as x3d

Será utilizado como exemplo o escoamento ao redor de um cilindro como [banco de dados](https://github.com/fschuch/xcompact3d_toolbox_data).
Definindo `cache=True` pode-se definir um local para salvar os dados no seu computador, `cache_dir="./example/"`.

In [2]:
cylinder_ds, prm = x3d.tutorial.open_dataset("cylinder", cache=True, cache_dir="./example/")

Nesse momento é bom verificar as informações do banco de dados.

In [ ]:
cylinder_ds.info()

Os dados são organizados em um [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) com as variáveis `u` (campo de velocidade), `pp` (pressão) e `epsi` (descrição da geométria), suas coordenadas (`x`, `y`, `t` e `i`) e alguns atributos como a versão do Xcompact3d e a `url` onde pode-se encontrar o banco de dados.

No próximo bloco se configura a toolbox e alguns atributos do banco de dados, para que se possa salvar todos os arquivos binários no disco. Os comandos a baixo seram descritos melhor adiante.

In [ ]:
x3d.param["mytype"] = np.float32

prm.dataset.set(data_path="./data/", drop_coords="z")

cylinder_ds.u.attrs["file_name"] = "u"
cylinder_ds.pp.attrs["file_name"] = "pp"
cylinder_ds.epsi.attrs["file_name"] = "epsilon"

prm.write("input.i3d")

prm.dataset.write(cylinder_ds)

prm.dataset.write_xdmf("xy-planes.xdmf")

del cylinder_ds, prm

Após essas linhas de código os arquivos são organizados dessa forma:

```
tutorial
│   computing_and_plotting.ipynb
│   io.ipynb
│   input.i3d
│   parameters.ipynb
│   xy-planes.xdmf
│
└─── data
│       │   epsilon.bin
│       │   pp-000.bin
│       │   pp-001.bin
│       │   ...
│       │   pp-199.bin
│       │   pp-200.bin
│       │   ux-000.bin
│       │   ux-001.bin
│       │   ...
│       │   ux-199.bin
│       │   ux-200.bin
│       │   uy-000.bin
│       │   uy-001.bin
│       │   ...
│       │   uy-199.bin
│       │   uy-200.bin
│       │   uz-000.bin
│       │   uz-001.bin
│       │   ...
│       │   uz-199.bin
│       │   uz-200.bin
│
└─── example
│       │   cylinder.nc
```

É bem próximo o que é gerado após rodar a simulação.

## Por que xarray?

As estruturas de dados são providas por [xarray](http://docs.xarray.dev/en/stable/index.html), que atribui rótulos de dimensão, coordenadas e atributos sob NumPy arrays. E utiliza [dask](https://dask.org/) para computação paralela.
O objetivo é acelerar o desenvolvimento e customização do pós-processamento.

Adicionamente, xcompact3d-toolbox prove funcionalidades para [DataArray](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataArray) e [Dataset](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataset).

Mais detalhes da biblioteca [xarray](http://docs.xarray.dev/en/stable/index.html) estão disponíveis em [Overview: Why xarray?](http://docs.xarray.dev/en/stable/getting-started-guide/why-xarray.html) e [Quick overview](http://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html).

## Objetos xarray sobre demanda

Como primeiro etapa do pós-processamento, se carrega o arquivo com os parâmetros da simulação.

In [5]:
prm = x3d.Parameters(loadfile="input.i3d")

Para salvar espaço no disco, o banco de dados foi convertido de precisão dupla para precisão simples, isso pode ser feito com:

In [6]:
x3d.param["mytype"] = np.float32

A ferramenta permite ler arquivos com diferentes extensões. Para mais detalhes acesse [filename properties](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.FilenameProperties). Para carregar o banco de dados baixado, pode-se confirgurar a ferramenta com o comando a baixo:

In [7]:
prm.dataset.filename_properties.set(
    separator="-",
    file_extension=".bin",
    number_of_digits=3,
)

Agora se especifica os parâmetros do banco de dados, como a localização do arquivo, se alguma coordenada pode ser ignorada (`drop_coords` permite excluir uma coordenada, como exemplo, excluir a coordenada `z` para trabalhar com o plano `xy`), o número de passos de tempo `snapshot_counting` e o tamnho do passo de tempo`snapshot_step`. Para mais detalhes acesse a documentação do módulo [dataset](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset).
Para esse exemplo se define os seguintes parâmetros:

In [8]:
prm.dataset.set(
    data_path="./data/",
    drop_coords="z",
    snapshot_counting="ilast",
    snapshot_step="ioutput",
)

Pode-se checar o [tamanho do banco de dados](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.__len__) com:

In [ ]:
len(prm.dataset)

Isso significa que há 201 snapshots indo de 0 (i.g., `ux-000.bin`) a 200 (i.g., `ux-200.bin`).

Pode-se carregar um arquivo binário com:

In [10]:
epsilon = prm.dataset.load_array("./data/epsilon.bin", add_time=False)

Observe que [load_array](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.load_array) necessita o caminho completo e se usa `add_time=False` porque esse array não envolve tempo.

Pode-se visualizar na tela:

In [ ]:
epsilon

Fazendo a mesma coisa, agora para o campo de velocidade na direção x, `ux`, e usando `add_time=True`:

In [12]:
ux = prm.dataset.load_array("./data/ux-100.bin", add_time=True)

Agora `t` é uma coordenada e seu valor adimensional é automaticamente calculado como `75.0`:

In [ ]:
ux

Também pode-se carregar a [serie temporal](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.load_time_series) completa para uma variável, se houver espaço suficiente na memória, ou simplesmente por:

In [ ]:
ux = prm.dataset["ux"]

Pode-se visualizar o banco de dados na tela com:

In [ ]:
ux

Pode-se carregar cada array em uma variável diferente:

In [ ]:
ux = prm.dataset["ux"]
uy = prm.dataset["uy"]
pp = prm.dataset["pp"]

Ou organizar vários arrays em um banco de dados:

In [ ]:
# create an empty dataset
ds = xr.Dataset()

# populate it
for var in ["ux", "uy", "pp"]:
    ds[var] = prm.dataset[var]

# show on the screen
ds

A mesma solução pode ser escrita em uma única linha de código:

```python
ds = xr.Dataset({var: prm.dataset[var] for var in "ux uy pp".split()})
```

É possível carregar todas as variáveis de um único snapshot com [load_snapshot](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.load_snapshot) ou:

In [18]:
snapshot = prm.dataset[100]

Se obtém um [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) com todas as variáveis com as suas coordenadas. Cada variável pode ser acessada individualmente com a notação por ponto (i.g., `snapshot.pp`, `snapshot.ux`, `snapshot.uy`) ou por colchete (i.g., `snapshot["pp"]`, `snapshot["ux"]`, `snapshot["uy"]`).

In [ ]:
snapshot

Pode-se usar um [slice](https://docs.python.org/3/library/functions.html#slice) para selecionar uma seção do banco de dados para calcular a [média temporal](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.mean.html):

In [ ]:
time_averaged = prm.dataset[-100:].mean("t")
time_averaged

Também pode-se carregar todos o snapshots com a seguinte notação:

In [ ]:
prm.dataset[:]

Para o caso que não aja memória suficiente para carregar toda simulação, pode-se iterar sobre todos os snapshots, um por um:

In [22]:
for ds in prm.dataset:
    # Calcular a vorticidade como exemplo
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

Outra opção viável é usar `reversed(prm.dataset)` para iterar na ordem inversa.

Também pode-se selecionar um intervalo de snapshots carregados, um por um:

In [23]:
for ds in prm.dataset(100, 200, 1):
    # Calcular a vorticidade como exemplo
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

In [ ]:
# Resultado para última interação
vort

## Salvando resultados em um arquivo binário

No último exemplo, foi calculado a vorticidade do escoamento, porém os dados foram perdidos ao final do loop. Para salvar o resultado poderia-se escrever os dados no disco usando [write](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.io.Dataset.write).

In [25]:
for ds in prm.dataset:
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")
    prm.dataset.write(data=vort, file_prefix="w3")

O exemplo acima utiliza [xarray.DataArray](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray).  Pode-se fazer uma implementação equivalente com [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset). Todavia, a uma diferença importante, apenas os arrays com o atributo chamado `file_name` serão salvos. Isso é feito para evitar que os arquivos originais de (`ux`, `uy`, `uz`, ...) fossem reescritos por acidente.

Reescrevendo o exemplo anterior para salvar `vort` no banco de dados `ds`. Definindo um atributo `file_name` para `w3`, os arrays serão salvos como `w3-000.bin`, `w3-001.bin`, `w3-002.bin`, etc.

Warnings serão suprimidos já que a aplicação irá alertar que `pp`, `ux` e `uy` não serão salvos por não ter um `file_name`. Porém, não é esse o objetivo.

In [26]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    for ds in prm.dataset:
        ds["vort"] = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")
        ds["vort"].attrs["file_name"] = "w3"
        prm.dataset.write(ds)

O método [prm.dataset.write()](https://xcompact3d-toolbox.readthedocs.io/en/stable/references/api-reference.html#xcompact3d_toolbox.io.Dataset.write) salva os arquivos binários da mesma forma que [XCompact3d](https://github.com/xcompact3d/Incompact3d) faria. Isso permite que o arquivo seja carregado pelo solver numérico e também pode-se processá-lo com qualquer outra ferramenta já usada com o Xcompact3d.

Como exemplo, carregando snapshot 0, `w3` é carregado também:

In [ ]:
prm.dataset[0]

### Atualizando o arquivo xdmf

Após computar e salvar os novos resultados no disco, para que se possa ser salvo em qualquer ferramenta externa, como ParaView. Para atualizar o arquivo xdmf com a vorticidade `w3`, pode-se usar a linha de código:

In [ ]:
prm.dataset.write_xdmf("xy-planes.xdmf")

## Outros formatos

Objetos xarray podem ser exportados para outros formatos dependendo da necessidade.

Por exemplo, [xarray.DataArray](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray) e [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) pode ser salva com [netCDF](http://docs.xarray.dev/en/stable/user-guide/io.html). Dessa forma mantendo as dimensões, coordenadas e atributos.

Só para ter uma estimativa sobre o espaço ocupado no disco, o tamanho do banco de dados `cylinder.nc` baixado para esse tutorial é 75.8 MB. O tamanho da pasta `./data/` depois de gerar osa arquivos binários da mesma forma que [XCompact3d](https://github.com/xcompact3d/Incompact3d) deve ser 75.7 MB.

Como exemplo de uso do formato netCDF, toma-se um snapshot:

In [ ]:
snapshot = prm.dataset[0]
snapshot

Pode-se adicionar informações extras para quem acessar esse banco de dados. Atributos podem ser incluidos para cada array, coordenada e atributos globais para o banco de dados. Eles são salvos em um dicionário.

Veja exemplo:

In [30]:
# Definindo o atributo para cada coordenada
snapshot.x.attrs = {"name": "x", "long_name": "Stream-wise coordinate", "units": "-"}
snapshot.y.attrs = {"name": "y", "long_name": "Vertical coordinate", "units": "-"}
snapshot.t.attrs = {"name": "t", "long_name": "Time", "units": "-"}

# Definindo o atributo para cada array
snapshot.ux.attrs = {"name": "ux", "long_name": "Stream-wise velocity", "units": "-"}
snapshot.uy.attrs = {"name": "y", "long_name": "Vertical velocity", "units": "-"}
snapshot.pp.attrs = {"name": "p", "long_name": "Pressure", "units": "-"}
snapshot.w3.attrs = {"name": "w3", "long_name": "Vorticity", "units": "-"}

# Definindo os atributos para o banco de dados
snapshot.attrs = {
    "title": "An example from the tutorials",
    "url": "https://docs.fschuch.com/xcompact3d_toolbox/tutorial/io.html",
    "authors": "List of names",
    "doi": "maybe a fancy doi from zenodo",
}

Exportado como arquivo netCDF:

In [31]:
snapshot.to_netcdf("snapshot-000.nc")

Importando o arquivo netCDF:

In [32]:
snapshot_in = xr.open_dataset("snapshot-000.nc")

Observe o resultado, mantém todas as dimensões, coordenadas e atributos:

In [ ]:
snapshot_in

Pode-se comparar os dois banco de dados para verificar se os dados, dimensões e coordenadas são as mesmas:

In [34]:
xr.testing.assert_equal(snapshot, snapshot_in)

Xarray é desenvolvido sobre Numpy, o objeto `numpy.ndarray`  pode ser acessado com a propriedade `values` (i.g., `epsilon.values`). É compatível com `numpy.save` e outros métodos disponibilizados do ecosistema Numpy/SciPy (Não seja necessário usar explicitamente `.values`). COmo no exemplo:

In [ ]:
np.save("epsi.npy", epsilon)
epsi_in = np.load("epsi.npy")

print(type(epsi_in))
epsi_in

Pode ser usado com ferramentas de pós-processamento previo.

Se o tamanho do banco de dados for reduzido com os métodos [integrate](https://docs.xarray.dev/en/stablegenerated/xarray.Dataset.integrate.html), [mean](https://docs.xarray.dev/en/stablegenerated/xarray.Dataset.mean.html), ou [sel](https://docs.xarray.dev/en/stablegenerated/xarray.Dataset.sel.html) em um subconjuntos de dados, pode converte-lo para [pandas.Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) e exportá-lo CSV, Excel, etc.

Como exemplo, selecionou-se um perfil vertical para todas as variáveis onde `x = 20` e [converte-la em um dataframe](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.to_dataframe.html):

In [ ]:
snapshot_in.sel(x=20.0).to_dataframe()

Para mais informações acesse [documentação do pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) para mais detalhes.